<a href="https://colab.research.google.com/github/MyPham1207/Python_for_ML/blob/main/Assignments/Clustering%20click%20points%20on%20map/Day3_clustering_within_district.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.Load libraries


In [ ]:
!pip install --upgrade pip
!pip install --upgrade numpy

In [ ]:
!pip install osmnet
!pip install matplotlib==3.1.3
!pip install osmnet
!pip install pandana
!pip install folium
!pip install rtree
!pip install pygeos

In [ ]:
import folium
from folium import plugins

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import json
import requests

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

from geopandas.tools import sjoin
import geopandas

#2.Load data

In [ ]:
%cd drive/My Drive/Python_for_ML/Assignment_2

In [ ]:
! git clone https://github.com/CityScope/CSL_HCMC

##2.1.Load top 5 pop_increase district

In [ ]:
district = geopandas.read_file('CSL_HCMC/Data/GIS/Population/population_HCMC/population_shapefile/Population_District_Level.shp')
district

In [ ]:
district = district.to_crs("EPSG:3857")

In [ ]:
district_top5 = district[['Dist_Name']]
district_top5['pop_increase'] = (district['Pop_2019']/district['Pop_2017']).to_frame()
district_top5 = district_top5.nlargest(5, 'pop_increase')
district_top5

In [ ]:
district = district[district['Dist_Name'].isin(district_top5['Dist_Name'])]
district

##2.2.Load click data

In [ ]:
hits = geopandas.read_file('/content/drive/MyDrive/Python_for_ML/Assignment_1/Data/response.json')
hits = hits.to_crs('epsg:3857')
hits

In [ ]:
click_data = geopandas.sjoin(hits, district, op='within')
click_data

#3.KMeans clustering

In [ ]:
#Take coordination of click_data
click_data["longitude"] = click_data.geometry.map(lambda p: p.x)
click_data["latitude"] = click_data.geometry.map(lambda p: p.y)
click_points = click_data[['latitude', 'longitude', 'Dist_Name']]
click_points.groupby(by='Dist_Name')
click_points

In [ ]:
top5_district = list(set(click_points['Dist_Name']))
top5_district

In [ ]:
click_points['Dist_Name'].value_counts()

##3.1.Use Elbow method to choose k

In [ ]:
def elbowMethod(data):
  distortions = []
  K = range(20, 100, 10)
  for k in K:
    kmean = KMeans(n_clusters=k, random_state=42).fit(data)
    distortions.append(kmean.inertia_)
  plt.plot(K, distortions, 'bx-')
  plt.xlabel('k')
  plt.ylabel('Distortion')
  plt.title('The Elbow Method')
  plt.show()

In [ ]:
for dist in top5_district:
  temp = click_points[click_points['Dist_Name'] == dist]
  elbowMethod(temp[['latitude', 'longitude']])

In [ ]:
K = [50, 60, 50, 60, 60]#base on plot above
#teacher give K=30 so n_clusters for all district = 30
points = geopandas.GeoDataFrame()
center = []
for i in range(0, 5):
  #KMeans clustering for each district
  temp = click_points[click_points['Dist_Name'] == top5_district[i]]
  k_means = KMeans(n_clusters=30, random_state=42).fit(temp[['latitude', 'longitude']])
  temp['cluster'] = k_means.labels_

  #find largest cluster
  number_of_occurences = pd.DataFrame(temp['cluster'].value_counts())
  number_of_occurences.reset_index(inplace=True)
  number_of_occurences.columns = ['Cluster', 'Count']
  top = number_of_occurences['Cluster'][0]
  center.append(list(k_means.cluster_centers_[top]))
  temp = temp[temp['cluster'] == top]

  points = pd.concat([points, temp])

center

#4.Heat map

In [ ]:
gdf = geopandas.points_from_xy(points.longitude, points.latitude, crs="EPSG:3857")
gdf = gdf.to_crs(epsg=4326)
points['geometry'] = gdf
geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in points.geometry ]
geo_df_list

In [ ]:
centroid = geopandas.points_from_xy([row[1] for row in center], [row[0] for row in center], crs="EPSG:3857")
centroid = centroid.to_crs(epsg=4326)
centroid = [[point.xy[1][0], point.xy[0][0]] for point in centroid ]
centroid

In [ ]:
colors = ['red', 'darkblue', 'black', 'green', 'purple']

In [ ]:
map = folium.Map(location = geo_df_list[0], zoom_start = 12)
plugins.HeatMap(geo_df_list).add_to(map)
for i in range(5):
  folium.Marker(centroid[i],icon=folium.Icon(color=colors[i])).add_to(map)
map